In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import os
os.chdir("/content/drive/My Drive/data")
import pandas as pd
import numpy as np
dat = pd.read_csv("final_data0723_3.csv", encoding='CP949')

In [ ]:
dat = dat[('2019-07-02'<=dat['date']) & (dat['date']<'2020-06-30')]. dropna()

In [ ]:
! pip install catboost

     |████████████████████████████████| 64.8MB 64kB/s 


In [ ]:
from catboost import CatBoostRegressor
cb_model = CatBoostRegressor(iterations=1000,
                          loss_function='RMSE',
                          eval_metric='RMSE',#task_type="GPU"
)

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True, random_state=0)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True, random_state=0)

In [ ]:
day_index=[]
for i in range(int(dat.shape[0]/24)):
  day_index .append(i)

In [ ]:
tr_idx =[]
te_idx =[]
for i, (tr_day_index, te_day_index) in enumerate(kf.split(day_index)):
  tr_idx.append(tr_day_index)
  te_idx.append(te_day_index)

In [ ]:
tr_idx

[array([  0,   1,   2,   3,   4,   8,   9,  10,  11,  12,  13,  14,  16,
         17,  18,  19,  21,  22,  23,  24,  25,  27,  28,  29,  30,  31,
         32,  33,  34,  35,  36,  38,  39,  40,  41,  42,  43,  44,  46,
         47,  48,  49,  50,  51,  53,  57,  58,  61,  62,  63,  66,  67,
         69,  70,  71,  72,  73,  75,  77,  78,  79,  80,  81,  82,  83,
         84,  85,  86,  87,  88,  91,  92,  93,  94,  95,  96,  97,  98,
         99, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 114,
        115, 117, 118, 119, 121, 123, 124, 125, 126, 127, 128, 129, 130,
        131, 132, 134, 135, 137, 138, 139, 140, 141, 142, 143, 144, 146,
        147, 148, 149, 150, 151, 152, 154, 155, 156, 157, 160, 161, 163,
        164, 165, 166, 168, 169, 171, 172, 173, 174, 175, 176, 177, 178,
        180, 181, 182, 183, 184, 185, 187, 188, 189, 192, 193, 194, 195,
        197, 199, 201, 202, 203, 204, 207, 209, 211, 212, 213, 215, 219,
        220, 221, 222, 225, 226, 227, 228, 229, 231

In [ ]:
te_idx

[array([  5,   6,   7,  15,  20,  26,  37,  45,  52,  54,  55,  56,  59,
         60,  64,  65,  68,  74,  76,  89,  90, 100, 101, 113, 116, 120,
        122, 133, 136, 145, 153, 158, 159, 162, 167, 170, 179, 186, 190,
        191, 196, 198, 200, 205, 206, 208, 210, 214, 216, 217, 218, 223,
        224, 230, 234, 237, 239, 248, 255, 259, 260, 263, 264, 266, 289,
        295, 299, 300, 301, 305, 306, 322, 342]),
 array([  1,   8,  12,  17,  21,  22,  33,  34,  46,  49,  63,  66,  73,
         78,  81,  85,  92,  97, 102, 103, 106, 108, 114, 124, 126, 132,
        134, 135, 140, 141, 142, 144, 146, 150, 171, 176, 181, 213, 215,
        221, 229, 233, 238, 241, 249, 254, 268, 269, 272, 275, 278, 280,
        281, 282, 287, 296, 312, 313, 318, 324, 325, 327, 329, 338, 340,
        345, 349, 351, 352, 353, 355, 360]),
 array([  4,  16,  18,  27,  29,  35,  44,  61,  67,  71,  75,  79,  83,
         93,  96, 109, 110, 111, 118, 129, 137, 138, 139, 149, 152, 154,
        156, 157, 160, 161, 1

In [ ]:
fold_train=[]
for i in range(5):
  fold = []
  for j in range(len(tr_idx[i])):
    for k in range(24):
      tmp = tr_idx[i][j] * 24 + k 
      fold.append(tmp)
  fold_train.append(list(fold))

In [ ]:
fold_test=[]
for i in range(5):
  fold = []
  for j in range(len(te_idx[i])):
    for k in range(24):
      tmp = te_idx[i][j] * 24 + k 
      fold.append(tmp)
  fold_test.append(list(fold))

In [ ]:
def cv_cat(train):
  result = []
  X = train[train.columns.difference(['y'])]
  y = train.loc[:, 'y']
  for i in range(5):
    train_index = fold_train[i]
    test_index = fold_test[i]
    y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
    X_train, X_valid = X.iloc[train_index,:], X.iloc[test_index,:]
    cat_fit = cb_model.fit(X_train, y_train, cat_features=list(train.select_dtypes(include=['object', 'int64']).columns))
    pred = cat_fit.predict(X_valid)
    true = y_valid
    error = mean_squared_error(true, pred, squared=False)
    result.append(error)
  return (np.mean(result)) 


In [ ]:
X = dat[dat.columns.difference(['y'])]
y = dat.loc[:, 'y']
train_index = fold_train[i]
test_index = fold_test[i]


In [ ]:
cv_cat(dat)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
5:	learn: 20.5850000	total: 220ms	remaining: 36.5s
6:	learn: 19.6313669	total: 244ms	remaining: 34.6s
7:	learn: 18.7454902	total: 267ms	remaining: 33.1s
8:	learn: 17.9371207	total: 291ms	remaining: 32.1s
9:	learn: 17.1467950	total: 312ms	remaining: 30.9s
10:	learn: 16.3884008	total: 333ms	remaining: 30s
11:	learn: 15.7073545	total: 346ms	remaining: 28.5s
12:	learn: 15.0510792	total: 370ms	remaining: 28.1s
13:	learn: 14.4355619	total: 392ms	remaining: 27.6s
14:	learn: 13.8698858	total: 405ms	remaining: 26.6s
15:	learn: 13.3086457	total: 428ms	remaining: 26.3s
16:	learn: 12.8198354	total: 450ms	remaining: 26s
17:	learn: 12.3376076	total: 472ms	remaining: 25.8s
18:	learn: 11.9059629	total: 497ms	remaining: 25.7s
19:	learn: 11.5181398	total: 514ms	remaining: 25.2s
20:	learn: 11.1488003	total: 523ms	remaining: 24.4s
21:	learn: 10.7854332	total: 546ms	remaining: 24.3s
22:	learn: 10.4384825	total: 570ms	remaining: 24.2s
23:	learn: 10.1086272	total: 593ms	re

5.506235404502599

In [ ]:
#variable_list = list(dat.columns)
#variable = [e for e in variable_list if e not in ('y','date','day','wday','ymd' )]
variable = ['month', 'hour', 'temp', 'rain', 'wind_speed', 'humidity', 'rain_prob', 'rain_type', 'skycondition', 'daily', 'daily_smooth', 'pm10_cat', 'pm2_5_cat', 'altitude', 'azimuth', 'sin_time', 'cos_time', 'temp_diff']

In [ ]:
from itertools import combinations

In [ ]:
train = dat.dropna()
#train = dat.iloc[:100,]

In [ ]:
i = len(variable)
best_df =pd.DataFrame()
while i>5:
  variable_df = pd.DataFrame()
  temp_comb = list(combinations(variable, i))
  variable_df ['variable'] = temp_comb
  variable_df['rmse'] =0
  for k in range(len(temp_comb)):
    var_name = list(temp_comb[k])
    var = var_name + ['y']
    train_cv = train.loc[:,var]
    variable_df.iloc[k, 1] = cv_cat(train_cv)
 
  idx = np.where(min(variable_df['rmse']))
  min_variable = variable_df.iloc[idx].iloc[0,0]
  best_df = best_df.append(variable_df.iloc[idx])
  best_df.to_csv('/content/drive/My Drive/data/backward_stepwise_add.csv')
  variable = list(min_variable)
  i=i-1
  
 

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
188:	learn: 5.4203645	total: 8.53s	remaining: 36.6s
189:	learn: 5.4179483	total: 8.57s	remaining: 36.5s
190:	learn: 5.4157040	total: 8.62s	remaining: 36.5s
191:	learn: 5.4122127	total: 8.66s	remaining: 36.4s
192:	learn: 5.4063290	total: 8.7s	remaining: 36.4s
193:	learn: 5.4017940	total: 8.75s	remaining: 36.3s
194:	learn: 5.3996167	total: 8.79s	remaining: 36.3s
195:	learn: 5.3946543	total: 8.84s	remaining: 36.2s
196:	learn: 5.3937822	total: 8.88s	remaining: 36.2s
197:	learn: 5.3913453	total: 8.93s	remaining: 36.2s
198:	learn: 5.3882132	total: 8.97s	remaining: 36.1s
199:	learn: 5.3872978	total: 9.03s	remaining: 36.1s
200:	learn: 5.3824045	total: 9.07s	remaining: 36.1s
201:	learn: 5.3799021	total: 9.13s	remaining: 36s
202:	learn: 5.3775818	total: 9.18s	remaining: 36s
203:	learn: 5.3745351	total: 9.21s	remaining: 36s
204:	learn: 5.3725781	total: 9.26s	remaining: 35.9s
205:	learn: 5.3708031	total: 9.31s	remaining: 35.9s
206:	learn: 5.3641483	total: 9.34s	

In [ ]:
best_idx = np.where(min(best_df['rmse']))
print('best variable:',best_df.iloc[best_idx].iloc[0,0])